# Pandas 데모 - Explarotary Data Analysis

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

In [59]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures # 데이터 표준화

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools
from sklearn.metrics import log_loss

In [4]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 데이터 다운로드

데이터는 [Dacon 단국대 소/중 데이터 분석 AI 경진대회 웹사이트](https://www.dacon.io/competitions/official/235638/data/)에서 다운로드 받아 `../input` 폴더에 저장.

In [5]:
!ls -alF ../input/

total 4408604
drwxr-xr-x  2 swcu swcu       4096  9월 29 19:09 ./
drwxrwxr-x 10 swcu swcu       4096 10월  5 21:59 ../
-rw-r--r--  1 swcu swcu   47899736  9월 18 02:22 ecminer_standard_normal_outlier_train.csv
-rw-r--r--  1 swcu swcu  132585197  9월 24 15:52 feature.csv
-rw-r--r--  1 swcu swcu     720009  9월 23 11:34 lgbm_u_r_i_z_redshift_du_dg_dr_di_dz.csv
-rw-r--r--  1 swcu swcu     800010  7월 26 13:35 sample_submission.csv
-rw-r--r--  1 swcu swcu   17619545  9월 18 01:17 standard_normal_test.xlsx
-rw-r--r--  1 swcu swcu   18936246  7월 26 13:35 test.csv
-rw-r--r--  1 swcu swcu   76278443  7월 26 13:35 train.csv
-rw-r--r--  1 swcu swcu 3360982474  9월 30 10:56 trn_engineering.csv
-rw-r--r--  1 swcu swcu  858552178  9월 30 10:57 tst_engineering.csv


In [6]:
data_dir = Path('../input/')
trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = data_dir / 'sample_submission.csv'
submission_dir = Path('../submission')
target_col = 'class'
seed = 42

In [7]:
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

#feature_file = feature_dir / f'{feature_name}.csv'
submission_file = submission_dir / f'{model_name}.csv'

## EDA

In [8]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(320000, 19)


,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0


## 데이터 개요

In [9]:
trn.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,3.2000e+05,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000
mean,19.8524,18.3768,17.7358,17.1742,16.8752,5.8308e-02,19.6289,18.2271,17.6154,17.1523,16.8693,6.3327,6.1388,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9455,31.1108,1.4662,52.5615,57.1463,5.7546e-01,1.9236,17.7866,1.4730,25.0758,35.4365,8.8811,8.5516,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,-17.6875,-17555.3656,7.7314,-23955.8178,-23955.8178,-1.6605e+02,-42.8375,-9999.0000,-16.3802,-9999.0000,-9999.0000,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2889,3.5580e-05,18.5636,17.3484,16.7871,16.4530,16.2341,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4194,18.1405,17.5258,17.1497,16.9176,4.7129e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4321,19.0732,18.4281,18.0074,17.7289,9.4597e-02,20.1976,18.8884,18.2911,17.9070,17.6557,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [10]:
trn.dtypes

u            float64
g            float64
r            float64
i            float64
z            float64
redshift     float64
dered_u      float64
dered_g      float64
dered_r      float64
dered_i      float64
dered_z      float64
nObserve       int64
nDetect        int64
airmass_u    float64
airmass_g    float64
airmass_r    float64
airmass_i    float64
airmass_z    float64
class          int64
dtype: object

In [11]:
for i in trn.columns:
    print(i, len(trn.loc[trn[i]<0]))


u 3
g 4
r 0
i 4
z 9
redshift 70876
dered_u 8
dered_g 9
dered_r 6
dered_i 7
dered_z 8
nObserve 0
nDetect 0
airmass_u 0
airmass_g 0
airmass_r 0
airmass_i 0
airmass_z 0
class 0


In [12]:
col = ['u', 'g', 'r', 'i', 'z', 'dered_u', 'dered_g', 
      'dered_r', 'dered_i', 'dered_z']
trn1 = trn.copy()
for i in col:
    trn1 = trn1[trn1[i] >=0]

trn1

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,18.3656,17.2409,16.7739,16.4850,16.3404,5.1988e-02,18.1328,17.0406,16.6374,16.3861,16.2874,2,2,1.0930,1.0888,1.0973,1.0951,1.0909,2
319996,18.6856,17.6658,17.1904,16.8169,16.6367,8.7261e-02,18.5207,17.5131,17.0871,16.7436,16.6017,2,2,1.0517,1.0508,1.0526,1.0522,1.0513,1
319997,21.1890,18.4234,17.2956,16.9011,16.6841,-1.0148e-05,20.9890,18.3076,17.2152,16.8403,16.6355,31,30,1.1933,1.1941,1.1925,1.1929,1.1937,0


In [13]:
trn1.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,3.1998e+05,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000
mean,19.8528,18.4320,17.7358,17.3043,17.0613,5.8309e-02,19.6301,18.2593,17.6161,17.2154,16.9947,6.3328,6.1390,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9428,1.6591,1.4662,1.3198,1.3333,5.7548e-01,1.9072,1.6566,1.4665,1.3249,1.3233,8.8814,8.5519,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,2.2651,1.2383,7.7314,2.7492,5.1281,-1.6605e+02,0.7538,3.0828,6.0403,6.8561,7.2567,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2890,3.5580e-05,18.5637,17.3484,16.7873,16.4531,16.2342,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4195,18.1405,17.5258,17.1498,16.9177,4.7133e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0729,18.4281,18.0075,17.7290,9.4600e-02,20.1976,18.8885,18.2912,17.9070,17.6556,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [14]:
tst = pd.read_csv(tst_file, index_col = 0)
tst

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
id,,,,,,,,,,,,,,,,,,
320000,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,19.0750,17.9550,17.4308,17.0838,16.9087,1,1,1.1563,1.1515,1.1613,1.1588,1.1539
320001,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,15.0703,14.3098,14.3901,14.5160,14.6372,2,2,1.0076,1.0074,1.0080,1.0078,1.0075
320002,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,19.1004,17.7726,17.1460,16.7497,16.5689,1,1,1.3809,1.3905,1.3715,1.3761,1.3856
320003,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,17.9729,17.0735,16.8470,16.7700,16.7668,1,1,1.1408,1.1408,1.1409,1.1408,1.1408
320004,17.8325,16.7646,16.2135,15.8722,15.6621,6.8471e-02,17.7545,16.6794,16.1563,15.8315,15.6433,2,2,1.1110,1.1077,1.1145,1.1127,1.1093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885


In [15]:
df = pd.concat([trn1, tst])
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861,NaN
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878,NaN
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885,NaN


# Feature Engineering

In [16]:
target_col = 'class'
feature_col = [x for x in trn1.columns if trn1[x].dtype in [np.int64, np.float64] and x !=target_col]
len(feature_col)

18

In [17]:
oneChar_col = ['u','g','r','i','z']
dered_col = [c for c in df.columns if c.find('dered') != -1]
airmass_col = [c for c in df.columns if c.find('airmass') != -1]
print(dered_col)
print(airmass_col)

['dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z']
['airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z']


In [18]:
# #polynomial 2 추가

scaler = StandardScaler()
poly = PolynomialFeatures(2)
poly_X = poly.fit_transform(scaler.fit_transform(df.drop(target_col, axis=1)))
feature_names = poly.get_feature_names(df.columns)
feature_names

['1',
 'u',
 'g',
 'r',
 'i',
 'z',
 'redshift',
 'dered_u',
 'dered_g',
 'dered_r',
 'dered_i',
 'dered_z',
 'nObserve',
 'nDetect',
 'airmass_u',
 'airmass_g',
 'airmass_r',
 'airmass_i',
 'airmass_z',
 'u^2',
 'u g',
 'u r',
 'u i',
 'u z',
 'u redshift',
 'u dered_u',
 'u dered_g',
 'u dered_r',
 'u dered_i',
 'u dered_z',
 'u nObserve',
 'u nDetect',
 'u airmass_u',
 'u airmass_g',
 'u airmass_r',
 'u airmass_i',
 'u airmass_z',
 'g^2',
 'g r',
 'g i',
 'g z',
 'g redshift',
 'g dered_u',
 'g dered_g',
 'g dered_r',
 'g dered_i',
 'g dered_z',
 'g nObserve',
 'g nDetect',
 'g airmass_u',
 'g airmass_g',
 'g airmass_r',
 'g airmass_i',
 'g airmass_z',
 'r^2',
 'r i',
 'r z',
 'r redshift',
 'r dered_u',
 'r dered_g',
 'r dered_r',
 'r dered_i',
 'r dered_z',
 'r nObserve',
 'r nDetect',
 'r airmass_u',
 'r airmass_g',
 'r airmass_r',
 'r airmass_i',
 'r airmass_z',
 'i^2',
 'i z',
 'i redshift',
 'i dered_u',
 'i dered_g',
 'i dered_r',
 'i dered_i',
 'i dered_z',
 'i nObserve',
 '

In [19]:
poly_X.shape

(399978, 190)

In [20]:
poly_df = pd.DataFrame(poly_X, columns = ['poly_'+ x for x in feature_names], index = df.index)
poly_df

,poly_1,poly_u,poly_g,poly_r,poly_i,poly_z,poly_redshift,poly_dered_u,poly_dered_g,poly_dered_r,poly_dered_i,poly_dered_z,poly_nObserve,poly_nDetect,poly_airmass_u,poly_airmass_g,poly_airmass_r,poly_airmass_i,poly_airmass_z,poly_u^2,poly_u g,poly_u r,poly_u i,poly_u z,poly_u redshift,poly_u dered_u,poly_u dered_g,poly_u dered_r,poly_u dered_i,poly_u dered_z,poly_u nObserve,poly_u nDetect,poly_u airmass_u,poly_u airmass_g,poly_u airmass_r,poly_u airmass_i,poly_u airmass_z,poly_g^2,poly_g r,poly_g i,poly_g z,poly_g redshift,poly_g dered_u,poly_g dered_g,poly_g dered_r,poly_g dered_i,poly_g dered_z,poly_g nObserve,poly_g nDetect,poly_g airmass_u,...,poly_dered_r airmass_u,poly_dered_r airmass_g,poly_dered_r airmass_r,poly_dered_r airmass_i,poly_dered_r airmass_z,poly_dered_i^2,poly_dered_i dered_z,poly_dered_i nObserve,poly_dered_i nDetect,poly_dered_i airmass_u,poly_dered_i airmass_g,poly_dered_i airmass_r,poly_dered_i airmass_i,poly_dered_i airmass_z,poly_dered_z^2,poly_dered_z nObserve,poly_dered_z nDetect,poly_dered_z airmass_u,poly_dered_z airmass_g,poly_dered_z airmass_r,poly_dered_z airmass_i,poly_dered_z airmass_z,poly_nObserve^2,poly_nObserve nDetect,poly_nObserve airmass_u,poly_nObserve airmass_g,poly_nObserve airmass_r,poly_nObserve airmass_i,poly_nObserve airmass_z,poly_nDetect^2,poly_nDetect airmass_u,poly_nDetect airmass_g,poly_nDetect airmass_r,poly_nDetect airmass_i,poly_nDetect airmass_z,poly_airmass_u^2,poly_airmass_u airmass_g,poly_airmass_u airmass_r,poly_airmass_u airmass_i,poly_airmass_u airmass_z,poly_airmass_g^2,poly_airmass_g airmass_r,poly_airmass_g airmass_i,poly_airmass_g airmass_z,poly_airmass_r^2,poly_airmass_r airmass_i,poly_airmass_r airmass_z,poly_airmass_i^2,poly_airmass_i airmass_z,poly_airmass_z^2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.7530,1.1414,0.8667,0.2786,-0.0126,-0.1093,1.8290,1.2046,0.9114,0.3138,-0.0038,1.3135,1.3865,0.1200,0.1195,0.1204,0.1202,0.1198,3.0731,2.0009,1.5194,0.4883,-0.0220,-0.1916,3.2062,2.1116,1.5978,0.5501,-0.0067,2.3027,2.4306,0.2104,0.2095,0.2110,0.2108,0.2100,1.3028,0.9893,0.3180,-0.0143,-0.1247,2.0876,1.3749,1.0403,0.3582,-0.0044,1.4993,1.5826,0.1370,...,0.1094,0.1089,0.1097,0.1096,0.1092,0.0985,-0.0012,0.4122,0.4351,0.0377,0.0375,0.0378,0.0377,0.0376,1.4679e-05,-0.0050,-0.0053,-0.0005,-0.0005,-0.0005,-0.0005,-0.0005,1.7254,1.8213,0.1577,0.1570,0.1581,0.1579,0.1574,1.9225,0.1664,0.1657,0.1669,0.1667,0.1661,0.0144,0.0143,0.0144,0.0144,0.0144,1.4282e-02,0.0144,0.0144,0.0143,0.0145,0.0145,0.0144,0.0145,0.0144,0.0144
1,1.0,-2.4688,-2.6206,-2.9171,-3.0577,-0.3527,-0.1007,-2.4423,-2.5661,-2.8639,-3.0004,-0.2413,-0.6010,-0.6014,0.6658,0.6882,0.6414,0.6539,0.6773,6.0950,6.4697,7.2017,7.5489,0.8708,0.2487,6.0295,6.3353,7.0704,7.4074,0.5958,1.4836,1.4847,-1.6438,-1.6990,-1.5834,-1.6143,-1.6721,6.8675,7.6445,8.0131,0.9243,0.2640,6.4002,6.7248,7.5052,7.8629,0.6324,1.5749,1.5760,-1.7449,...,-1.9069,-1.9709,-1.8368,-1.8727,-1.9397,9.0025,0.7241,1.8031,1.8044,-1.9978,-2.0648,-1.9244,-1.9619,-2.0321,5.8236e-02,0.1450,0.1451,-0.1607,-0.1661,-0.1548,-0.1578,-0.1634,0.3611,0.3614,-0.4001,-0.4136,-0.3854,-0.3930,-0.4070,0.3617,-0.4004,-0.4138,-0.3857,-0.3932,-0.4073,0.4433,0.4582,0.4270,0.4354,0.4510,4.7358e-01,0.4414,0.4500,0.4661,0.4114,0.4194,0.4344,0.4276,0.4429,0.4587
2,1.0,-1.5772,-1.5633,-1.4981,-1.4478,-0.1560,-0.1082,-1.5830,-1.5518,-1.4824,-1.4281,-0.1053,-0.4883,-0.4844,-1.3179,-1.2910,-1.3437,-1.3309,-1.3046,2.4876,2.4657,2.3629,2.2835,0.2460,0.1707,2.4967,2.4476,2.3380,2.2524,0.1661,0.7702,0.7641,2.0786,2.0362,2.1192,2.0991,2.0576,2.4440,2.3421,2.2635,0.2438,0.1692,2.4747,2.4261,2.3174,2.2326,0.1646,0.7634,0.7573,2.0603,...,1.9536,1.9138,1.9918,1.9729,1.9339,2.0394,0.1504,0.6974,0.6918,1.8821,1.8437,1.9189,1.9007,1.8631,1.1092e-02,0.0514,0.0510,0.1388,0.1360,0.1415,0.1402,0.1374,0.2385,0.2366,0.6436,0.6305,0.6562,0.6499,0.6371,0.2347,0.6384,0.6254,0.6509,0.6448,0.6320,1.7368,1.7015,1.7708,1.7540,1.719

In [21]:
#모든 조합 차이 feature
diff_feature_df = pd.DataFrame()
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_diff'
    diff_feature_df[new_c] = df[c1] - df[c2]


diff_feature_df

,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,g_airmass_i_diff,g_airmass_z_diff,r_i_diff,r_z_diff,r_redshift_diff,r_dered_u_diff,r_dered_g_diff,r_dered_r_diff,r_dered_i_diff,r_dered_z_diff,r_nObserve_diff,r_nDetect_diff,r_airmass_u_diff,r_airmass_g_diff,r_airmass_r_diff,r_airmass_i_diff,r_airmass_z_diff,i_z_diff,i_redshift_diff,...,dered_g_airmass_u_diff,dered_g_airmass_g_diff,dered_g_airmass_r_diff,dered_g_airmass_i_diff,dered_g_airmass_z_diff,dered_r_dered_i_diff,dered_r_dered_z_diff,dered_r_nObserve_diff,dered_r_nDetect_diff,dered_r_airmass_u_diff,dered_r_airmass_g_diff,dered_r_airmass_r_diff,dered_r_airmass_i_diff,dered_r_airmass_z_diff,dered_i_dered_z_diff,dered_i_nObserve_diff,dered_i_nDetect_diff,dered_i_airmass_u_diff,dered_i_airmass_g_diff,dered_i_airmass_r_diff,dered_i_airmass_i_diff,dered_i_airmass_z_diff,dered_z_nObserve_diff,dered_z_nDetect_diff,dered_z_airmass_u_diff,dered_z_airmass_g_diff,dered_z_airmass_r_diff,dered_z_airmass_i_diff,dered_z_airmass_z_diff,nObserve_nDetect_diff,nObserve_airmass_u_diff,nObserve_airmass_g_diff,nObserve_airmass_r_diff,nObserve_airmass_i_diff,nObserve_airmass_z_diff,nDetect_airmass_u_diff,nDetect_airmass_g_diff,nDetect_airmass_r_diff,nDetect_airmass_i_diff,nDetect_airmass_z_diff,airmass_u_airmass_g_diff,airmass_u_airmass_r_diff,airmass_u_airmass_i_diff,airmass_u_airmass_z_diff,airmass_g_airmass_r_diff,airmass_g_airmass_i_diff,airmass_g_airmass_z_diff,airmass_r_airmass_i_diff,airmass_r_airmass_z_diff,airmass_i_airmass_z_diff
0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,19.1474,19.1466,1.3371,2.0699,19.0096,-4.1148,-1.2483,0.0544,1.3774,2.1006,1.0095,1.0095,17.8197,17.8188,17.8205,17.8201,17.8193,0.7328,17.6725,...,19.0680,19.0671,19.0688,19.0684,19.0676,1.3230,2.0462,0.9551,0.9551,17.7654,17.7644,17.7661,17.7658,17.7649,0.7232,-0.3679,-0.3679,16.4423,16.4414,16.4431,16.4428,16.4419,-1.0911,-1.0911,15.7192,15.7183,15.7200,15.7196,15.7187,0,16.8102,16.8093,16.8110,16.8106,16.8098,16.8102,16.8093,16.8110,16.8106,16.8098,-9.1700e-04,7.8500e-04,4.0900e-04,-4.4200e-04,1.7020e-03,1.3260e-03,4.7500e-04,-3.7600e-04,-1.2270e-03,-8.5100e-04
1,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,12.8110,12.8064,0.1840,0.2835,13.4479,-1.5140,-0.5521,0.0410,0.2162,0.3177,12.4524,12.4524,12.1992,12.1946,12.2037,12.2014,12.1969,0.0995,13.2639,...,12.7513,12.7467,12.7558,12.7535,12.7490,0.1751,0.2767,12.4114,12.4114,12.1581,12.1536,12.1626,12.1604,12.1559,0.1016,12.2363,12.2363,11.9830,11.9784,11.9875,11.9853,11.9807,12.1347,12.1347,11.8814,11.8768,11.8859,11.8837,11.8791,0,-0.2533,-0.2578,-0.2488,-0.2510,-0.2555,-0.2533,-0.2578,-0.2488,-0.2510,-0.2555,-4.5820e-03,4.4920e-03,2.2570e-03,-2.2790e-03,9.0740e-03,6.8390e-03,2.3030e-03,-2.2350e-03,-6.7710e-03,-4.5360e-03
2,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,14.8037,14.8021,0.1428,0.1863,15.5358,-1.0713,-0.1503,0.0963,0.2146,0.2402,13.5363,13.5363,14.5138,14.5122,14.5153,14.5146,14.5130,0.0435,15.3931,...,14.6641,14.6625,14.6656,14.6649,14.6633,0.1183,0.1440,13.4400,13.4400,14.4175,14.4159,14.4191,14.4183,14.4167,0.0

In [22]:
# 모든 조합 나누기    
divide_feature_df = pd.DataFrame()
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_divide'
    divide_feature_df[new_c] = df[c1] / df[c2]

divide_feature_df

,u_g_divide,u_r_divide,u_i_divide,u_z_divide,u_redshift_divide,u_dered_u_divide,u_dered_g_divide,u_dered_r_divide,u_dered_i_divide,u_dered_z_divide,u_nObserve_divide,u_nDetect_divide,u_airmass_u_divide,u_airmass_g_divide,u_airmass_r_divide,u_airmass_i_divide,u_airmass_z_divide,g_r_divide,g_i_divide,g_z_divide,g_redshift_divide,g_dered_u_divide,g_dered_g_divide,g_dered_r_divide,g_dered_i_divide,g_dered_z_divide,g_nObserve_divide,g_nDetect_divide,g_airmass_u_divide,g_airmass_g_divide,g_airmass_r_divide,g_airmass_i_divide,g_airmass_z_divide,r_i_divide,r_z_divide,r_redshift_divide,r_dered_u_divide,r_dered_g_divide,r_dered_r_divide,r_dered_i_divide,r_dered_z_divide,r_nObserve_divide,r_nDetect_divide,r_airmass_u_divide,r_airmass_g_divide,r_airmass_r_divide,r_airmass_i_divide,r_airmass_z_divide,i_z_divide,i_redshift_divide,...,dered_g_airmass_u_divide,dered_g_airmass_g_divide,dered_g_airmass_r_divide,dered_g_airmass_i_divide,dered_g_airmass_z_divide,dered_r_dered_i_divide,dered_r_dered_z_divide,dered_r_nObserve_divide,dered_r_nDetect_divide,dered_r_airmass_u_divide,dered_r_airmass_g_divide,dered_r_airmass_r_divide,dered_r_airmass_i_divide,dered_r_airmass_z_divide,dered_i_dered_z_divide,dered_i_nObserve_divide,dered_i_nDetect_divide,dered_i_airmass_u_divide,dered_i_airmass_g_divide,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide
0,1.1439,1.2238,1.3164,1.3733,-2.8690e+05,1.0060,1.1484,1.2273,1.3194,1.3758,1.2924,1.2924,19.5534,19.5384,19.5663,19.5601,19.5462,1.0698,1.1508,1.2005,-2.5080e+05,0.8795,1.0039,1.0729,1.1534,1.2027,1.1298,1.1298,17.0931,17.0800,17.1044,17.0990,17.0868,1.0757,1.1222,-2.3444e+05,0.8221,0.9384,1.0029,1.0781,1.1242,1.0561,1.0561,15.9775,15.9652,15.9881,15.9830,15.9716,1.0433,-2.1795e+05,...,17.0267,17.0136,17.0380,17.0326,17.0204,1.0750,1.1210,1.0531,1.0531,15.9318,15.9196,15.9424,15.9373,15.9259,1.0428,0.9796,0.9796,14.8198,14.8084,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993
1,1.0704,1.1189,1.1344,1.1430,3.3404e+03,1.0057,1.0748,1.1223,1.1372,1.1460,15.0521,15.0521,12.0105,11.9667,12.0537,12.0321,11.9887,1.0453,1.0598,1.0678,3.1207e+03,0.9396,1.0041,1.0485,1.0624,1.0706,14.0620,14.0620,11.2204,11.1795,11.2607,11.2406,11.2000,1.0139,1.0215,2.9854e+03,0.8988,0.9606,1.0031,1.0163,1.0242,13.4524,13.4524,10.7340,10.6949,10.7726,10.7534,10.7145,1.0076,2.9446e+03,...,11.1745,11.1338,11.2147,11.1947,11.1542,1.0132,1.0211,13.4114,13.4114,10.7013,10.6623,10.7397,10.7206,10.6818,1.0077,13.2363,13.2363,10.5615,10.5230,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964
2,1.0607,1.0805,1.0905,1.0936,3.5566e+04,1.0108,1.0701,1.0872,1.0956,1.0974,8.3932,8.3932,16.4170,16.3913,16.4414,16.4294,16.4043,1.0186,1.0281,1.0310,3.3530e+04,0.9529,1.0089,1.0250,1.0329,1.0346,7.9127,7.9127,15.4772,15.4529,15.5002,15.4889,15.4652,1.0093,1.0121,3.2918e+04,0.9355,0.9904,1.0062,1.0140,1.0157,7.7682,7.7682,15.1945,15.1706,15.2170,15.2059,15.1827,1.00

In [23]:
# zip 함수를 이용한 row별, 시리즈 그룹별 max, min, max-min, std, sum 을 feature로 생성
# 그 외에도 mean, skew 등을 사용해 볼 수도 있긴 함 

for prefix, g in zip(['one','dered','airmass'], [oneChar_col, dered_col, airmass_col]):
    
    df[f'{prefix}_max'] = df[g].max(axis=1)
    
    df[f'{prefix}_min'] = df[g].min(axis=1)
    
    df[f'{prefix}_diff'] = df[f'{prefix}_max'] - df[f'{prefix}_min']
    
    df[f'{prefix}_std'] = df[g].std(axis=1)
    
    df[f'{prefix}_sum'] = df[g].sum(axis=1)
    
    

In [24]:
# 각 그룹별 max-max, min-min, sum-sum, std-std을 feature로 생성

for c in itertools.combinations(['one','dered','airmass'],2):
    df[f'{c[0]}_{c[1]}_max_diff'] = df[f'{c[0]}_max'] - df[f'{c[1]}_max']
    
    df[f'{c[0]}_{c[1]}_min_diff'] = df[f'{c[0]}_min'] - df[f'{c[1]}_min']
    
    df[f'{c[0]}_{c[1]}_sum_diff'] = df[f'{c[0]}_sum'] - df[f'{c[1]}_sum']
    
    df[f'{c[0]}_{c[1]}_std_diff'] = df[f'{c[0]}_std'] - df[f'{c[1]}_std']
    

In [25]:
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,25.6606,19.6346,6.0260,2.4968,106.6001,25.3536,19.5552,5.7985,2.4085,105.7854,1.2061,1.2049,1.1840e-03,4.6906e-04,6.0272,0.3070,0.0795,0.8147,0.0883,24.4546,18.4298,100.5730,2.4964,24.1476,18.3503,99.7582,2.4081
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0.0,24.4534,17.9826,6.4708,2.6489,100.5018,23.7714,17.8759,5.8955,2.4156,99.1344,1.1943,1.1937,5.4700e-04,2.1830e-04,5.9699,0.6820,0.1067,1.3673,0.2333,23.2591,16.7889,94.5319,2.6487,22.5771,16.6822,93.1645,2.4154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861,NaN,19.6734,18.0627,1.6107,0.6656,92.7910,19.3361,17.9670,1.3692,0.5751,91.7685,1.1863,1.1855,8.2900e-04,3.2915e-04,5.9294,0.3372,0.0957,1.0225,0.0905,18.4871,16.8772,86.8616,0.6653,18.1498,16.7815,85.8392,0.5748
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878,NaN,21.5229,20.6087,0.9142,0.3562,104.7105,21.3993,20.5451,0.8543,0.3364,104.3177,1.2908,1.2793,1.1443e-02,4.5229e-03,6.4250,0.1235,0.0636,0.3928,0.0198,20.2321,19.3294,98.2855,0.3516,20.1086,19.2657,97.8927,0.3319
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885,NaN,19.3991,18.1605,1.2386,0.5369,92.2002,18.9136,17.8753,1.0383,0.4336,90.7379,1.1886,1.1885,8.8000e-05,3.5436e-05,5.9425,0.4855,0.2852,1.4623,0.1033,18.2105,16.9720,86.2577,0.5368,17.7250,16.6869,84.7953,0.4336


In [26]:
total_df = pd.concat([df, poly_df, diff_feature_df, divide_feature_df], axis = 1)
total_df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,poly_1,poly_u,poly_g,poly_r,...,dered_g_airmass_u_divide,dered_g_airmass_g_divide,dered_g_airmass_r_divide,dered_g_airmass_i_divide,dered_g_airmass_z_divide,dered_r_dered_i_divide,dered_r_dered_z_divide,dered_r_nObserve_divide,dered_r_nDetect_divide,dered_r_airmass_u_divide,dered_r_airmass_g_divide,dered_r_airmass_r_divide,dered_r_airmass_i_divide,dered_r_airmass_z_divide,dered_i_dered_z_divide,dered_i_nObserve_divide,dered_i_nDetect_divide,dered_i_airmass_u_divide,dered_i_airmass_g_divide,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,1.0,1.7530,1.1414,0.8667,...,17.0267,17.0136,17.0380,17.0326,17.0204,1.0750,1.1210,1.0531,1.0531,15.9318,15.9196,15.9424,15.9373,15.9259,1.0428,0.9796,0.9796,14.8198,14.8084,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,1.0,-2.4688,-2.6206,-2.9171,...,11.1745,11.1338,11.2147,11.1947,11.1542,1.0132,1.0211,13.4114,13.4114,10.7013,10.6623,10.7397,10.7206,10.6818,1.0077,13.2363,13.2363,10.5615,10.5230,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,1.0,-1.5772,-1.5633,-1.4981,...,15.3414,15.3174,15.3642,15.3530,15.3295,1.0077,1.0094,7.7200,7.7200,15.1003,15.0766,15.1227,15.1117,15.0886,1.0017,7.6609,7.6609,14.9846,14.9611,15.0068,14.9959,14.9730,7.6480,7.6480,14.9595,14.9360,14.9817,14.9708,14.9479,1.0000,1.9560,1.9529,1.9589,1.9575,1.9545,1.9560,1.9529,1.958

In [27]:
df = total_df.copy()
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,poly_1,poly_u,poly_g,poly_r,...,dered_g_airmass_u_divide,dered_g_airmass_g_divide,dered_g_airmass_r_divide,dered_g_airmass_i_divide,dered_g_airmass_z_divide,dered_r_dered_i_divide,dered_r_dered_z_divide,dered_r_nObserve_divide,dered_r_nDetect_divide,dered_r_airmass_u_divide,dered_r_airmass_g_divide,dered_r_airmass_r_divide,dered_r_airmass_i_divide,dered_r_airmass_z_divide,dered_i_dered_z_divide,dered_i_nObserve_divide,dered_i_nDetect_divide,dered_i_airmass_u_divide,dered_i_airmass_g_divide,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,1.0,1.7530,1.1414,0.8667,...,17.0267,17.0136,17.0380,17.0326,17.0204,1.0750,1.1210,1.0531,1.0531,15.9318,15.9196,15.9424,15.9373,15.9259,1.0428,0.9796,0.9796,14.8198,14.8084,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,1.0,-2.4688,-2.6206,-2.9171,...,11.1745,11.1338,11.2147,11.1947,11.1542,1.0132,1.0211,13.4114,13.4114,10.7013,10.6623,10.7397,10.7206,10.6818,1.0077,13.2363,13.2363,10.5615,10.5230,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,1.0,-1.5772,-1.5633,-1.4981,...,15.3414,15.3174,15.3642,15.3530,15.3295,1.0077,1.0094,7.7200,7.7200,15.1003,15.0766,15.1227,15.1117,15.0886,1.0017,7.6609,7.6609,14.9846,14.9611,15.0068,14.9959,14.9730,7.6480,7.6480,14.9595,14.9360,14.9817,14.9708,14.9479,1.0000,1.9560,1.9529,1.9589,1.9575,1.9545,1.9560,1.9529,1.958

## Adding PCA 

In [28]:
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA

In [29]:
#default pca - 
temp = df.columns
temp = temp.drop('class')
pca = PCA(n_components=0.90)# 주성분 개수 설정 
X_pca = pca.fit_transform(df[temp])
print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_)

X_pca_t = X_pca.T

for i , ele in enumerate(X_pca_t):
    PCAn = 'PCA'+str(i)
    df[PCAn] = ele
df

eigen_value : [1.11660542e+16]
explained variance ratio : [0.99948469]


,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,poly_1,poly_u,poly_g,poly_r,...,dered_g_airmass_g_divide,dered_g_airmass_r_divide,dered_g_airmass_i_divide,dered_g_airmass_z_divide,dered_r_dered_i_divide,dered_r_dered_z_divide,dered_r_nObserve_divide,dered_r_nDetect_divide,dered_r_airmass_u_divide,dered_r_airmass_g_divide,dered_r_airmass_r_divide,dered_r_airmass_i_divide,dered_r_airmass_z_divide,dered_i_dered_z_divide,dered_i_nObserve_divide,dered_i_nDetect_divide,dered_i_airmass_u_divide,dered_i_airmass_g_divide,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA0
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,1.0,1.7530,1.1414,0.8667,...,17.0136,17.0380,17.0326,17.0204,1.0750,1.1210,1.0531,1.0531,15.9318,15.9196,15.9424,15.9373,15.9259,1.0428,0.9796,0.9796,14.8198,14.8084,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,1.0,-2.4688,-2.6206,-2.9171,...,11.1338,11.2147,11.1947,11.1542,1.0132,1.0211,13.4114,13.4114,10.7013,10.6623,10.7397,10.7206,10.6818,1.0077,13.2363,13.2363,10.5615,10.5230,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,1.0,-1.5772,-1.5633,-1.4981,...,15.3174,15.3642,15.3530,15.3295,1.0077,1.0094,7.7200,7.7200,15.1003,15.0766,15.1227,15.1117,15.0886,1.0017,7.6609,7.6609,14.9846,14.9611,15.0068,14.9959,14.9730,7.6480,7.6480,14.9595,14.9360,14.9817,14.9708,14.9479,1.0000,1.9560,1.9529,1.9589,1.9575,1.9545,1.9560,1.9529,1.9589,1.9575,1.9545,0.998

In [30]:
X_pca.shape

(399978, 1)

In [31]:
# incrementalPCA - difference is just dividing n_batches for memory 
n_batches = 1000
inc_pca = IncrementalPCA(n_components=36)#temp.size*0.5)
for batch_x in np.array_split(df[temp], n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)
    
X_inc = inc_pca.transform(df[temp])


# print('eigen_value :', inc_pca.explained_variance_)
# print('explained variance ratio :', inc_pca.explained_variance_ratio_)

# X_inc

X_inc_t = X_inc.transpose()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [32]:
for i , ele in enumerate(X_inc_t):
    incPCAn = 'incPCA'+str(i)
    df[incPCAn] = ele
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,poly_1,poly_u,poly_g,poly_r,...,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA0,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,incPCA18,incPCA19,incPCA20,incPCA21,incPCA22,incPCA23,incPCA24,incPCA25,incPCA26,incPCA27,incPCA28,incPCA29,incPCA30,incPCA31,incPCA32,incPCA33,incPCA34,incPCA35
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,1.0,1.7530,1.1414,0.8667,...,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3297,-2043.2987,-1.3828,-0.7607,-80.8455,20.0449,-8.8454,3.9333,-6.8195,-14.3186,-0.3939,-1.3279,-1.1551,-0.7940,0.0587,-0.7173,0.7374,-0.9695,-2.8444,1.8109,1.7376,-0.4724,-2.3203,-0.8141,-0.3559,1.6721,-1.2768,0.9250,0.9920,3.0288,-0.5918,-1.3182,-0.1658
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,1.0,-2.4688,-2.6206,-2.9171,...,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9779,-1.4115,-1.1955,28.6124,-54.9245,18.7223,23.2730,-36.6675,25.6147,-13.7076,8.1656,-4.2807,-3.3647,1.4848,-7.3249,1.4704,-2.2484,-7.9798,-1.1569,-2.5283,-0.2894,0.3329,0.6789,0.1551,-0.7529,-0.3361,-1.4448,0.2673,-0.5384,-0.3325,-0.7929,1.5427
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,1.0,-1.5772,-1.5633,-1.4981,...,1.9589,1.9575,1.9545,0.9984,1.0015,1.0008,0.9992,1.0031,1.0023,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-1.6850,-1.1089,16.5516,-27.1475,-9.8824,-3.2209,-13.5823,10.6947,13.4595,-3.2166,0.5658,-3.6758,-0.0442,-0.3687,-0.7381,-0.2475,-2.1062,-3.5442,-1.5270,-0.0281,0.1233,-0.2998,0.1454,1.2179,0.0403,0.2213,0.2558,-0.0663,0.0035,0.2189,-0.0581
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,25.6606,19.6346,6.0260,2.4968,106.6001,25.3536,19.5552,5.7985,2.4085,105.7854,1.2061,1.2049,1.1840e-03,4.6906e-04,6.0272,0.3070,0.0795,0.8147,0.0883,24.4546,18.4298,100.5730,2.4964,24.1476,18.3503,99.7582,2.4081,1.0,2.9852,1.6522,1.6917,...,2.4899,2.4894,2.4882,0.9995,1.0004,1.0002,0.9997,1.0010,1.0008,1.0003,0

# 훈련셋과 테스트셋 나누기

In [33]:
trn2 = df.iloc[:319978]
tst2 = df.iloc[319978:]
tst2.drop('class', axis =1 , inplace = True)
trn2.shape, tst2.shape

((319978, 579), (80000, 578))

# outlier

In [34]:
# 전체 데이터가 아닌 각 클래스별(0,1,2)로 이상치 처리
# Train_copy1 = Train_set.copy()

def remove_outlier(data, column):
    
    print(column)
    # 클래스 0처리
    column_data_0 = data[data['class']==0][column]
    quan_25_0 = np.percentile(column_data_0.values, 0.1) # 1분위수
    quan_75_0 = np.percentile(column_data_0.values, 99.9) # 3분위수
    iqr_0 = quan_75_0 - quan_25_0 # IQR 계산
    iqr_0 *= 2
    lowest_0 = quan_25_0 - iqr_0 # 최솟값 설정
    highest_0 = quan_75_0 + iqr_0 # 최댓값 설정
    outlier_index_0 = column_data_0[(column_data_0 < lowest_0) | (column_data_0 > highest_0)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_0))
    
    # 클래스 1처리
    column_data_1 = data[data['class']==1][column]
    quan_25_1 = np.percentile(column_data_1.values, 0.1) # 1분위수
    quan_75_1 = np.percentile(column_data_1.values, 99.9) # 3분위수
    iqr_1 = quan_75_1 - quan_25_1 # IQR 계산
    iqr_1 *= 2
    lowest_1 = quan_25_1 - iqr_1 # 최솟값 설정
    highest_1 = quan_75_1 + iqr_1 # 최댓값 설정
    outlier_index_1 = column_data_1[(column_data_1 < lowest_1 )| (column_data_1 > highest_1)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_1))
    
    # 클래스 2 처리
    column_data_2 = data[data['class']==2][column]
    quan_25_2 = np.percentile(column_data_2.values, 0.1) # 1분위수
    quan_75_2 = np.percentile(column_data_2.values, 99.9) # 3분위수
    iqr_2 = quan_75_2 - quan_25_2 # IQR 계산
    iqr_2 *= 2
    lowest_2 = quan_25_2 - iqr_2 # 최솟값 설정
    highest_2 = quan_75_2 + iqr_2 # 최댓값 설정
    outlier_index_2 = column_data_2[(column_data_2 < lowest_2 )| (column_data_2 > highest_2)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_2))
    # outlier 행 제거
    data.drop(list(set(list(outlier_index_0) + list(outlier_index_1) + list(outlier_index_2))), axis = 0, inplace = True)
    print(data.shape)
    return data

In [35]:
cols = list(trn2.columns)
cols.remove('redshift')
cols.remove('class')
# col

In [36]:
df2 = trn2.copy()
for i, col in enumerate(cols):
    print(i+'번째 칼럼')
    ROT = remove_outlier(df2, col)
    
    df2 = ROT

df2

u
3
0
0
(319975, 579)
g
0
0
1
(319974, 579)
r
0
0
1
(319973, 579)
i
1
0
0
(319972, 579)
z
0
0
1
(319971, 579)
dered_u
0
0
0
(319971, 579)
dered_g
0
0
0
(319971, 579)
dered_r
0
0
0
(319971, 579)
dered_i
0
0
0
(319971, 579)
dered_z
0
0
0
(319971, 579)
nObserve
0
0
0
(319971, 579)
nDetect
0
0
0
(319971, 579)
airmass_u
0
0
0
(319971, 579)
airmass_g
0
0
0
(319971, 579)
airmass_r
0
0
0
(319971, 579)
airmass_i
0
0
0
(319971, 579)
airmass_z
0
0
0
(319971, 579)
one_max
0
0
0
(319971, 579)
one_min
0
0
0
(319971, 579)
one_diff
0
2
1
(319968, 579)
one_std
0
1
0
(319967, 579)
one_sum
0
0
0
(319967, 579)
dered_max
0
0
0
(319967, 579)
dered_min
0
0
0
(319967, 579)
dered_diff
0
0
0
(319967, 579)
dered_std
0
0
0
(319967, 579)
dered_sum
0
0
0
(319967, 579)
airmass_max
0
0
0
(319967, 579)
airmass_min
0
0
0
(319967, 579)
airmass_diff
0
0
0
(319967, 579)
airmass_std
0
0
0
(319967, 579)
airmass_sum
0
0
0
(319967, 579)
one_dered_max_diff
0
4
10
(319953, 579)
one_dered_min_diff
11
10
20
(319912, 579)
one_dere

0
0
(319492, 579)
poly_airmass_g airmass_r
0
0
0
(319492, 579)
poly_airmass_g airmass_i
0
0
0
(319492, 579)
poly_airmass_g airmass_z
0
0
0
(319492, 579)
poly_airmass_r^2
0
0
0
(319492, 579)
poly_airmass_r airmass_i
0
0
0
(319492, 579)
poly_airmass_r airmass_z
0
0
0
(319492, 579)
poly_airmass_i^2
0
0
0
(319492, 579)
poly_airmass_i airmass_z
0
0
0
(319492, 579)
poly_airmass_z^2
0
0
0
(319492, 579)
u_g_diff
0
0
0
(319492, 579)
u_r_diff
0
0
0
(319492, 579)
u_i_diff
0
0
0
(319492, 579)
u_z_diff
0
0
0
(319492, 579)
u_redshift_diff
0
0
0
(319492, 579)
u_dered_u_diff
0
2
4
(319486, 579)
u_dered_g_diff
0
0
0
(319486, 579)
u_dered_r_diff
0
0
0
(319486, 579)
u_dered_i_diff
0
0
0
(319486, 579)
u_dered_z_diff
0
0
0
(319486, 579)
u_nObserve_diff
0
0
0
(319486, 579)
u_nDetect_diff
0
0
0
(319486, 579)
u_airmass_u_diff
0
0
0
(319486, 579)
u_airmass_g_diff
0
0
0
(319486, 579)
u_airmass_r_diff
0
0
0
(319486, 579)
u_airmass_i_diff
0
0
0
(319486, 579)
u_airmass_z_diff
0
0
0
(319486, 579)
g_r_diff
0
5
23
(3

0
0
(319143, 579)
i_dered_g_divide
0
0
0
(319143, 579)
i_dered_r_divide
0
2
5
(319136, 579)
i_dered_i_divide
0
1
0
(319135, 579)
i_dered_z_divide
0
0
0
(319135, 579)
i_nObserve_divide
0
0
0
(319135, 579)
i_nDetect_divide
0
0
0
(319135, 579)
i_airmass_u_divide
0
0
0
(319135, 579)
i_airmass_g_divide
0
0
0
(319135, 579)
i_airmass_r_divide
0
0
0
(319135, 579)
i_airmass_i_divide
0
0
0
(319135, 579)
i_airmass_z_divide
0
0
0
(319135, 579)
z_redshift_divide
1
0
0
(319134, 579)
z_dered_u_divide
0
0
0
(319134, 579)
z_dered_g_divide
0
0
1
(319133, 579)
z_dered_r_divide
0
0
4
(319129, 579)
z_dered_i_divide
0
0
2
(319127, 579)
z_dered_z_divide
0
0
0
(319127, 579)
z_nObserve_divide
0
0
0
(319127, 579)
z_nDetect_divide
0
0
0
(319127, 579)
z_airmass_u_divide
0
0
0
(319127, 579)
z_airmass_g_divide
0
0
0
(319127, 579)
z_airmass_r_divide
0
0
0
(319127, 579)
z_airmass_i_divide
0
0
0
(319127, 579)
z_airmass_z_divide
0
0
0
(319127, 579)
redshift_dered_u_divide
0
0
0
(319127, 579)
redshift_dered_g_divide
0
0

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,poly_1,poly_u,poly_g,poly_r,...,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA0,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,incPCA18,incPCA19,incPCA20,incPCA21,incPCA22,incPCA23,incPCA24,incPCA25,incPCA26,incPCA27,incPCA28,incPCA29,incPCA30,incPCA31,incPCA32,incPCA33,incPCA34,incPCA35
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,0.0017,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,1.0,1.7530,1.1414,0.8667,...,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3297,-2043.2987,-1.3828,-0.7607,-80.8455,20.0449,-8.8454,3.9333,-6.8195,-14.3186,-0.3939,-1.3279,-1.1551,-0.7940,0.0587,-0.7173,0.7374,-0.9695,-2.8444,1.8109,1.7376,-0.4724,-2.3203,-0.8141,-0.3559,1.6721,-1.2768,0.9250,0.9920,3.0288,-0.5918,-1.3182,-0.1658
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,0.0091,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,1.0,-2.4688,-2.6206,-2.9171,...,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9779,-1.4115,-1.1955,28.6124,-54.9245,18.7223,23.2730,-36.6675,25.6147,-13.7076,8.1656,-4.2807,-3.3647,1.4848,-7.3249,1.4704,-2.2484,-7.9798,-1.1569,-2.5283,-0.2894,0.3329,0.6789,0.1551,-0.7529,-0.3361,-1.4448,0.2673,-0.5384,-0.3325,-0.7929,1.5427
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,0.0031,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,1.0,-1.5772,-1.5633,-1.4981,...,1.9589,1.9575,1.9545,0.9984,1.0015,1.0008,0.9992,1.0031,1.0023,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-1.6850,-1.1089,16.5516,-27.1475,-9.8824,-3.2209,-13.5823,10.6947,13.4595,-3.2166,0.5658,-3.6758,-0.0442,-0.3687,-0.7381,-0.2475,-2.1062,-3.5442,-1.5270,-0.0281,0.1233,-0.2998,0.1454,1.2179,0.0403,0.2213,0.2558,-0.0663,0.0035,0.2189,-0.0581
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,25.6606,19.6346,6.0260,2.4968,106.6001,25.3536,19.5552,5.7985,2.4085,105.7854,1.2061,1.2049,0.0012,4.6906e-04,6.0272,0.3070,0.0795,0.8147,0.0883,24.4546,18.4298,100.5730,2.4964,24.1476,18.3503,99.7582,2.4081,1.0,2.9852,1.6522,1.6917,...,2.4899,2.4894,2.4882,0.9995,1.0004,1.0002,0.9997,1.0010,1.0008,1.0003,0.9998,0.9993,0.9

In [37]:
target_col = 'class'
feature_col = [x for x in df.columns if df[x].dtype in [np.int64, np.float64] and x !=target_col]
print(feature_col)
pred_col='pred'


['u', 'g', 'r', 'i', 'z', 'redshift', 'dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z', 'nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z', 'one_max', 'one_min', 'one_diff', 'one_std', 'one_sum', 'dered_max', 'dered_min', 'dered_diff', 'dered_std', 'dered_sum', 'airmass_max', 'airmass_min', 'airmass_diff', 'airmass_std', 'airmass_sum', 'one_dered_max_diff', 'one_dered_min_diff', 'one_dered_sum_diff', 'one_dered_std_diff', 'one_airmass_max_diff', 'one_airmass_min_diff', 'one_airmass_sum_diff', 'one_airmass_std_diff', 'dered_airmass_max_diff', 'dered_airmass_min_diff', 'dered_airmass_sum_diff', 'dered_airmass_std_diff', 'poly_1', 'poly_u', 'poly_g', 'poly_r', 'poly_i', 'poly_z', 'poly_redshift', 'poly_dered_u', 'poly_dered_g', 'poly_dered_r', 'poly_dered_i', 'poly_dered_z', 'poly_nObserve', 'poly_nDetect', 'poly_airmass_u', 'poly_airmass_g', 'poly_airmass_r', 'poly_airmass_i', 'poly_airmass_z', 'poly_u^2', 'poly_u g', 'poly_u r', 'poly_u i', 'poly

In [38]:
from sklearn.feature_selection import SelectFromModel
#LGBM 모델을 기반으로 속성들 선택
select = SelectFromModel(LGBMClassifier(max_depth=27, learning_rate = 0.007, n_estimators = 5000, boosting_type='gbrt',
                                       ))#, threshold = 0.3)
select.fit(df2[feature_col], df2[target_col])

train_sel = select.transform(df2[feature_col])

test_sel = select.transform(tst2[feature_col])

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


In [79]:
# SelectFromModel?

## 모델링

In [39]:
df2.shape

(318853, 579)

In [40]:
train_sel.shape

(318853, 169)

In [41]:
y = df2[target_col]
df2.drop(target_col, axis=1, inplace=True)

feature_name = df2.columns.tolist()
print(y.shape, df2.shape, tst2.shape)

(318853,) (318853, 578) (80000, 578)


In [42]:
X_trn, X_val, y_trn, y_val = train_test_split(train_sel, y, test_size=.01, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)

clf = lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=15000,
                         num_leaves=256,
                         learning_rate=0.003,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=seed,
                         max_depth = 27,
                         n_jobs=-1,
                         boosting_type = 'gbdt')
clf.fit(X_trn, y_trn,
        eval_set=[(X_val, y_val)],
        eval_metric='multiclass',
        early_stopping_rounds=15)

p_val = clf.predict(X_val)


print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

(315664, 169) (3189, 169) (315664,) (3189,)
[1]	valid_0's multi_logloss: 0.980537
[2]	valid_0's multi_logloss: 0.975866
[3]	valid_0's multi_logloss: 0.971233
[4]	valid_0's multi_logloss: 0.966627
[5]	valid_0's multi_logloss: 0.96208
[6]	valid_0's multi_logloss: 0.95758
[7]	valid_0's multi_logloss: 0.957617
[8]	valid_0's multi_logloss: 0.953144
[9]	valid_0's multi_logloss: 0.948703
[10]	valid_0's multi_logloss: 0.948766
[11]	valid_0's multi_logloss: 0.944321
[12]	valid_0's multi_logloss: 0.945102
[13]	valid_0's multi_logloss: 0.940719
[14]	valid_0's multi_logloss: 0.936365
[15]	valid_0's multi_logloss: 0.932028
[16]	valid_0's multi_logloss: 0.927729
[17]	valid_0's multi_logloss: 0.929183
[18]	valid_0's multi_logloss: 0.924954
[19]	valid_0's multi_logloss: 0.924979
[20]	valid_0's multi_logloss: 0.925231
[21]	valid_0's multi_logloss: 0.926724
[22]	valid_0's multi_logloss: 0.933006
[23]	valid_0's multi_logloss: 0.928864
[24]	valid_0's multi_logloss: 0.939656
[25]	valid_0's multi_logloss: 0

[209]	valid_0's multi_logloss: 0.845533
[210]	valid_0's multi_logloss: 0.842132
[211]	valid_0's multi_logloss: 0.83875
[212]	valid_0's multi_logloss: 0.83538
[213]	valid_0's multi_logloss: 0.832048
[214]	valid_0's multi_logloss: 0.835182
[215]	valid_0's multi_logloss: 0.831834
[216]	valid_0's multi_logloss: 0.828514
[217]	valid_0's multi_logloss: 0.829828
[218]	valid_0's multi_logloss: 0.831213
[219]	valid_0's multi_logloss: 0.827908
[220]	valid_0's multi_logloss: 0.829387
[221]	valid_0's multi_logloss: 0.830983
[222]	valid_0's multi_logloss: 0.82767
[223]	valid_0's multi_logloss: 0.824368
[224]	valid_0's multi_logloss: 0.821091
[225]	valid_0's multi_logloss: 0.817841
[226]	valid_0's multi_logloss: 0.814622
[227]	valid_0's multi_logloss: 0.816178
[228]	valid_0's multi_logloss: 0.812942
[229]	valid_0's multi_logloss: 0.809752
[230]	valid_0's multi_logloss: 0.806561
[231]	valid_0's multi_logloss: 0.808397
[232]	valid_0's multi_logloss: 0.805222
[233]	valid_0's multi_logloss: 0.80204
[234

[415]	valid_0's multi_logloss: 0.720538
[416]	valid_0's multi_logloss: 0.71786
[417]	valid_0's multi_logloss: 0.715216
[418]	valid_0's multi_logloss: 0.712559
[419]	valid_0's multi_logloss: 0.709934
[420]	valid_0's multi_logloss: 0.711333
[421]	valid_0's multi_logloss: 0.712701
[422]	valid_0's multi_logloss: 0.710064
[423]	valid_0's multi_logloss: 0.711402
[424]	valid_0's multi_logloss: 0.712793
[425]	valid_0's multi_logloss: 0.713978
[426]	valid_0's multi_logloss: 0.715246
[427]	valid_0's multi_logloss: 0.712601
[428]	valid_0's multi_logloss: 0.714557
[429]	valid_0's multi_logloss: 0.715943
[430]	valid_0's multi_logloss: 0.717321
[431]	valid_0's multi_logloss: 0.71466
[432]	valid_0's multi_logloss: 0.715975
[433]	valid_0's multi_logloss: 0.713328
[434]	valid_0's multi_logloss: 0.71445
[435]	valid_0's multi_logloss: 0.711798
[436]	valid_0's multi_logloss: 0.709167
[437]	valid_0's multi_logloss: 0.710618
[438]	valid_0's multi_logloss: 0.708004
[439]	valid_0's multi_logloss: 0.709302
[44

[621]	valid_0's multi_logloss: 0.606714
[622]	valid_0's multi_logloss: 0.608222
[623]	valid_0's multi_logloss: 0.609277
[624]	valid_0's multi_logloss: 0.607215
[625]	valid_0's multi_logloss: 0.60516
[626]	valid_0's multi_logloss: 0.603095
[627]	valid_0's multi_logloss: 0.603839
[628]	valid_0's multi_logloss: 0.604907
[629]	valid_0's multi_logloss: 0.606308
[630]	valid_0's multi_logloss: 0.604262
[631]	valid_0's multi_logloss: 0.605354
[632]	valid_0's multi_logloss: 0.603307
[633]	valid_0's multi_logloss: 0.604372
[634]	valid_0's multi_logloss: 0.602313
[635]	valid_0's multi_logloss: 0.603681
[636]	valid_0's multi_logloss: 0.601636
[637]	valid_0's multi_logloss: 0.599601
[638]	valid_0's multi_logloss: 0.597582
[639]	valid_0's multi_logloss: 0.595563
[640]	valid_0's multi_logloss: 0.593557
[641]	valid_0's multi_logloss: 0.594657
[642]	valid_0's multi_logloss: 0.596015
[643]	valid_0's multi_logloss: 0.594016
[644]	valid_0's multi_logloss: 0.592039
[645]	valid_0's multi_logloss: 0.592901
[

[827]	valid_0's multi_logloss: 0.477955
[828]	valid_0's multi_logloss: 0.478719
[829]	valid_0's multi_logloss: 0.477301
[830]	valid_0's multi_logloss: 0.478115
[831]	valid_0's multi_logloss: 0.476701
[832]	valid_0's multi_logloss: 0.477342
[833]	valid_0's multi_logloss: 0.475936
[834]	valid_0's multi_logloss: 0.474525
[835]	valid_0's multi_logloss: 0.475387
[836]	valid_0's multi_logloss: 0.47601
[837]	valid_0's multi_logloss: 0.474604
[838]	valid_0's multi_logloss: 0.475636
[839]	valid_0's multi_logloss: 0.474231
[840]	valid_0's multi_logloss: 0.472837
[841]	valid_0's multi_logloss: 0.471448
[842]	valid_0's multi_logloss: 0.470068
[843]	valid_0's multi_logloss: 0.470712
[844]	valid_0's multi_logloss: 0.469329
[845]	valid_0's multi_logloss: 0.467951
[846]	valid_0's multi_logloss: 0.468695
[847]	valid_0's multi_logloss: 0.469409
[848]	valid_0's multi_logloss: 0.468042
[849]	valid_0's multi_logloss: 0.468973
[850]	valid_0's multi_logloss: 0.469686
[851]	valid_0's multi_logloss: 0.468307
[

...

[14197]	valid_0's multi_logloss: 0.144279
[14198]	valid_0's multi_logloss: 0.144278
[14199]	valid_0's multi_logloss: 0.144278
[14200]	valid_0's multi_logloss: 0.144277
[14201]	valid_0's multi_logloss: 0.144278
[14202]	valid_0's multi_logloss: 0.144277
[14203]	valid_0's multi_logloss: 0.144276
[14204]	valid_0's multi_logloss: 0.144281
[14205]	valid_0's multi_logloss: 0.144281
[14206]	valid_0's multi_logloss: 0.14428
[14207]	valid_0's multi_logloss: 0.144278
[14208]	valid_0's multi_logloss: 0.144278
[14209]	valid_0's multi_logloss: 0.144277
[14210]	valid_0's multi_logloss: 0.144277
[14211]	valid_0's multi_logloss: 0.144286
[14212]	valid_0's multi_logloss: 0.144284
[14213]	valid_0's multi_logloss: 0.14429
[14214]	valid_0's multi_logloss: 0.14429
[14215]	valid_0's multi_logloss: 0.144293
[14216]	valid_0's multi_logloss: 0.144288
[14217]	valid_0's multi_logloss: 0.144287
[14218]	valid_0's multi_logloss: 0.144287
[14219]	valid_0's multi_logloss: 0.14429
[14220]	valid_0's multi_logloss: 0.144

[14393]	valid_0's multi_logloss: 0.144339
[14394]	valid_0's multi_logloss: 0.14434
[14395]	valid_0's multi_logloss: 0.144348
[14396]	valid_0's multi_logloss: 0.144344
[14397]	valid_0's multi_logloss: 0.144343
[14398]	valid_0's multi_logloss: 0.144352
[14399]	valid_0's multi_logloss: 0.144351
[14400]	valid_0's multi_logloss: 0.144349
[14401]	valid_0's multi_logloss: 0.144348
[14402]	valid_0's multi_logloss: 0.144352
[14403]	valid_0's multi_logloss: 0.144352
[14404]	valid_0's multi_logloss: 0.144354
[14405]	valid_0's multi_logloss: 0.144353
[14406]	valid_0's multi_logloss: 0.144352
[14407]	valid_0's multi_logloss: 0.144356
[14408]	valid_0's multi_logloss: 0.144366
[14409]	valid_0's multi_logloss: 0.144369
[14410]	valid_0's multi_logloss: 0.144368
[14411]	valid_0's multi_logloss: 0.144375
[14412]	valid_0's multi_logloss: 0.144374
[14413]	valid_0's multi_logloss: 0.144373
[14414]	valid_0's multi_logloss: 0.144374
[14415]	valid_0's multi_logloss: 0.144374
[14416]	valid_0's multi_logloss: 0.

[14589]	valid_0's multi_logloss: 0.144442
[14590]	valid_0's multi_logloss: 0.144441
[14591]	valid_0's multi_logloss: 0.144438
[14592]	valid_0's multi_logloss: 0.144444
[14593]	valid_0's multi_logloss: 0.144442
[14594]	valid_0's multi_logloss: 0.144452
[14595]	valid_0's multi_logloss: 0.144452
[14596]	valid_0's multi_logloss: 0.144453
[14597]	valid_0's multi_logloss: 0.144452
[14598]	valid_0's multi_logloss: 0.144456
[14599]	valid_0's multi_logloss: 0.144455
[14600]	valid_0's multi_logloss: 0.144459
[14601]	valid_0's multi_logloss: 0.144461
[14602]	valid_0's multi_logloss: 0.144459
[14603]	valid_0's multi_logloss: 0.144458
[14604]	valid_0's multi_logloss: 0.144462
[14605]	valid_0's multi_logloss: 0.144466
[14606]	valid_0's multi_logloss: 0.144465
[14607]	valid_0's multi_logloss: 0.144464
[14608]	valid_0's multi_logloss: 0.144463
[14609]	valid_0's multi_logloss: 0.144458
[14610]	valid_0's multi_logloss: 0.144457
[14611]	valid_0's multi_logloss: 0.144456
[14612]	valid_0's multi_logloss: 0

[14785]	valid_0's multi_logloss: 0.144574
[14786]	valid_0's multi_logloss: 0.144577
[14787]	valid_0's multi_logloss: 0.144576
[14788]	valid_0's multi_logloss: 0.144577
[14789]	valid_0's multi_logloss: 0.144577
[14790]	valid_0's multi_logloss: 0.144576
[14791]	valid_0's multi_logloss: 0.144578
[14792]	valid_0's multi_logloss: 0.144584
[14793]	valid_0's multi_logloss: 0.144587
[14794]	valid_0's multi_logloss: 0.144583
[14795]	valid_0's multi_logloss: 0.144582
[14796]	valid_0's multi_logloss: 0.144583
[14797]	valid_0's multi_logloss: 0.144589
[14798]	valid_0's multi_logloss: 0.144589
[14799]	valid_0's multi_logloss: 0.144588
[14800]	valid_0's multi_logloss: 0.144586
[14801]	valid_0's multi_logloss: 0.144587
[14802]	valid_0's multi_logloss: 0.144593
[14803]	valid_0's multi_logloss: 0.144595
[14804]	valid_0's multi_logloss: 0.144595
[14805]	valid_0's multi_logloss: 0.144594
[14806]	valid_0's multi_logloss: 0.144593
[14807]	valid_0's multi_logloss: 0.144594
[14808]	valid_0's multi_logloss: 0

[14982]	valid_0's multi_logloss: 0.144659
[14983]	valid_0's multi_logloss: 0.144659
[14984]	valid_0's multi_logloss: 0.144658
[14985]	valid_0's multi_logloss: 0.144657
[14986]	valid_0's multi_logloss: 0.144655
[14987]	valid_0's multi_logloss: 0.144654
[14988]	valid_0's multi_logloss: 0.144653
[14989]	valid_0's multi_logloss: 0.144651
[14990]	valid_0's multi_logloss: 0.144651
[14991]	valid_0's multi_logloss: 0.144652
[14992]	valid_0's multi_logloss: 0.144653
[14993]	valid_0's multi_logloss: 0.144652
[14994]	valid_0's multi_logloss: 0.144651
[14995]	valid_0's multi_logloss: 0.14465
[14996]	valid_0's multi_logloss: 0.144659
[14997]	valid_0's multi_logloss: 0.144652
[14998]	valid_0's multi_logloss: 0.14465
[14999]	valid_0's multi_logloss: 0.144648
[15000]	valid_0's multi_logloss: 0.144647
94.2302%


In [43]:
print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')
p_tst = clf.predict(test_sel)

94.2302%


In [44]:
#submission
Submission_set = pd.read_csv(sample_file, index_col=0)

In [45]:
Submission_set['class'] = p_tst
print(Submission_set.shape)
Submission_set.head()

(80000, 1)


,class
id,
320000,2.0
320001,0.0
320002,2.0
320003,0.0
320004,2.0


In [46]:
#class 값 카운팅
Submission_set[target_col].value_counts()

2.0    40725
0.0    29969
1.0     9306
Name: class, dtype: int64

In [47]:
#저장하기
Submission_set.to_csv('../sub/lgb_3rd_updated.csv')


# HyperParameter Tuning

In [54]:
trn_df = pd.DataFrame(train_sel)
trn_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168
0,-8.1086e-05,6.3243,6.2153,2.4562,0.0017,6.7348e-04,0.1397,0.0307,0.3441,0.0432,21.9336,-0.1093,3.0731e+00,2.0009,-0.1916,3.2062,1.3028e+00,-0.1247,2.0876,-0.0947,0.0776,-0.0304,0.0002,0.0014,-0.0039,4.8121e-05,0.0119,-0.1999,-0.1316,-0.0996,-0.0343,0.0004,-0.1435,-0.1515,-0.0131,-0.0132,3.3451,2.2031,2.4024,1.4510,0.0985,-0.0012,1.4679e-05,0.0143,0.0145,2.9272,4.2545,3.0062,4.3088,1.3273,...,-4.0027e-06,-4.7955e-06,-4.5048e-06,-4.5048e-06,-6.8153e-05,-6.8101e-05,1.1415,1.2199,1.0687,1.1489,1.1981,1.0750,1.1210,1.0428,1.0000,4.1012e+05,-44588.1645,700.0365,-705.3297,-2043.2987,-1.3828,-0.7607,20.0449,-8.8454,3.9333,-6.8195,-14.3186,-0.3939,-1.3279,-1.1551,-0.7940,0.0587,-0.7173,0.7374,-0.9695,-2.8444,1.8109,1.7376,-0.4724,-2.3203,-0.8141,-0.3559,1.6721,-1.2768,0.9250,0.9920,3.0288,-0.5918,-1.3182,-0.1658
1,4.5061e-03,1.8832,1.8317,0.7586,0.0091,3.5867e-03,0.0857,0.0343,0.2507,0.0222,13.7086,-0.1007,6.0950e+00,6.4697,0.2487,6.0295,6.8675e+00,0.2640,6.4002,0.2939,9.3497,0.3080,0.1244,0.0355,1.0583,8.5116e-02,0.0101,0.2460,0.2585,0.2885,0.3023,0.0243,0.0605,0.0606,-0.0693,-0.0646,5.9647,6.2672,1.4677,6.5850,9.0025,0.7241,5.8236e-02,0.4736,0.4114,0.9902,1.5997,1.0476,1.6408,0.6096,...,3.2176e-04,3.4307e-04,4.5061e-03,4.5061e-03,3.5955e-03,3.5824e-03,1.0687,1.1159,1.0442,1.0580,1.0662,1.0132,1.0211,1.0077,1.0000,-1.3453e+05,1329.3993,234.1073,-139.3329,141.9779,-1.4115,-1.1955,-54.9245,18.7223,23.2730,-36.6675,25.6147,-13.7076,8.1656,-4.2807,-3.3647,1.4848,-7.3249,1.4704,-2.2484,-7.9798,-1.1569,-2.5283,-0.2894,0.3329,0.6789,0.1551,-0.7529,-0.3361,-1.4448,0.2673,-0.5384,-0.3325,-0.7929,1.5427
2,4.7198e-04,1.4364,1.3116,0.5462,0.0031,1.2343e-03,0.1787,0.0540,0.5396,0.0472,15.5835,-0.1082,2.4876e+00,2.4657,0.1707,2.4967,2.4440e+00,0.1692,2.4747,0.1622,2.0963,0.1567,0.0243,0.0169,0.2227,1.6425e-02,0.0117,0.1714,0.1680,0.1605,0.1546,0.0114,0.0529,0.0524,0.1398,0.1454,2.5058,2.4565,0.7730,2.4082,2.0394,0.1504,1.1092e-02,1.6668,1.8054,0.9610,1.2501,1.0998,1.3463,0.2891,...,3.0088e-05,3.0856e-05,2.3599e-04,2.3599e-04,4.6159e-04,4.6087e-04,1.0587,1.0756,1.0160,1.0238,1.0255,1.0077,1.0094,1.0017,1.0000,-2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-1.6850,-1.1089,-27.1475,-9.8824,-3.2209,-13.5823,10.6947,13.4595,-3.2166,0.5658,-3.6758,-0.0442,-0.3687,-0.7381,-0.2475,-2.1062,-3.5442,-1.5270,-0.0281,0.1233,-0.2998,0.1454,1.2179,0.0403,0.2213,0.2558,-0.0663,0.0035,0.2189,-0.0581
3,5.8143e-06,6.0260,5.7985,2.4085,0.0012,4.6906e-04,0.3070,0.0795,0.8147,0.0883,24.1476,-0.1091,8.9114e+00,4.9321,-0.3257,8.9443,2.7297e+00,-0.1803,4.9503,-0.1846,3.8504,-0.2141,0.0532,-0.0252,0.4483,3.7507e-02,0.0119,-0.3269,-0.1799,-0.1836,-0.2121,-0.0178,0.0287,0.0401,-0.0272,-0.0282,8.9772,4.9404,-0.7883,2.7188,3.7800,0.3163,2.6465e-02,0.0624,0.0670,4.4719,5.4395,4.6660,5.5734,0.9676,...,2.7694e-07,2.9733e-07,1.4536e-06,1.9381e-06,4.8235e-06,4.8209e-06,1.2076,1.2622,1.0452,1.0606,1.0736,1.0148,1.0272,1.0122,1.3333,-8.3498e+06,540693.7038,-255606.5334,48177.6402,58140.3173,-1.6736,-0.8981,48.7216,-31.1316,12.5749,-6.4611,-2.8258,-2.7074,-5.8254,-4.1219,3.3348,0.7668,-9.0993,1.1187,8.4260,-2.5833,-10.4184,1.4502,1.6020,-0.3707,-3.5396,-0.9125,-0.1803,1.6063,-2.2506,1.2199,-0.6800,-0.5546,1.0699,-0.0565
4,-3.3247e-05,6.4708,5.8955,2.4156,0.0005,2.1830e-04,0.6820,0.1067,1.3673,0.2333,22.5771,-0.1092,5.5911e+00,3.2126,-0.2582,5.1258,1.8460e+00,-0.1483,2.9453,-0.0971,0.5964,-0.0843,0.0066,-0.0089,0.0599,4.6485e-03,0.0119,-0.2367,-0.1431,-0.0927,-0.0802,-0.0062,-0.0819,-0.0748,-0.0164,-0.0177,4.6993,2.8413,1.6268,1.7180,0.5397,0.0419,3.2506e-03,0.0225,0.0262,3.7543,5.4111,4.0196,5.5904,1.6568,...,-1.6271e-06,-1.8599e-06,-2.5575e-0

In [55]:
y

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
319995    2.0
319996    1.0
319997    0.0
319998    0.0
319999    0.0
Name: class, Length: 318853, dtype: float64

In [56]:
import optuna
features = trn.columns
n_fold = 10

In [57]:
def objective(trial ,rounds):
    n_class = 3
    on_gpu=0
    p_val = np.zeros((trn_df.shape[0], n_class))
    
    X_trn, X_val, y_trn, y_val = train_test_split(trn_df, y, test_size=.01, random_state=seed)
  # for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
    param = {"objective": "multiclass",
          'learning_rate' : trial.suggest_loguniform('learning_rate',1e-3,1e-2),
          'num_iterations':trial.suggest_int('num_iterations',2000,20000),           
          'n_estimators': trial.suggest_int('n_estimators',5000,20000),
          'num_leaves': trial.suggest_int('num_leaves', 16, 512),
          'max_depth': trial.suggest_int('max_depth',10,30),
          'boosting_type':'gbdt',
          'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
          'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
          'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
          'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
          'metric':  'multi_logloss',
          'num_class': n_class,
          'verbose':-1}

    dtrain = lgb.Dataset(X_trn, label=y_trn)
    dvalid = lgb.Dataset(X_val, label=y_val)
  #print(f'training model for CV #{i}')
# pruning_callback = optuna.integration.LightGBMPruningCallback(trial, '')
    clf = lgb.train(param,
                  dtrain,
                  valid_sets=[dtrain,dvalid],
                  early_stopping_rounds=rounds,
#                  callbacks=[pruning_callback],
                  verbose_eval = 100
                  )
    
    p_val = clf.predict(X_val)
    loss = log_loss(y_val,p_val)
  
    print('best_score', clf.best_score)
    print(f'multi logloss: {loss}')

# print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')
    return loss

In [ ]:
study0 = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study0.optimize(lambda trial: objective(trial,100), n_trials=100)

[I 2020-10-08 14:46:45,965] A new study created in memory with name: no-name-1f7fc671-58f9-49e8-95a8-0211774efd68


Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.748028	valid_1's multi_logloss: 0.747708
[200]	training's multi_logloss: 0.59109	valid_1's multi_logloss: 0.591696
[300]	training's multi_logloss: 0.480789	valid_1's multi_logloss: 0.482362
[400]	training's multi_logloss: 0.40058	valid_1's multi_logloss: 0.403304
[500]	training's multi_logloss: 0.340945	valid_1's multi_logloss: 0.344824
[600]	training's multi_logloss: 0.295822	valid_1's multi_logloss: 0.300912
[700]	training's multi_logloss: 0.261226	valid_1's multi_logloss: 0.267518
[800]	training's multi_logloss: 0.234398	valid_1's multi_logloss: 0.241901
[900]	training's multi_logloss: 0.213313	valid_1's multi_logloss: 0.22216
[1000]	training's multi_logloss: 0.196595	valid_1's multi_logloss: 0.206806
[1100]	training's multi_logloss: 0.183195	valid_1's multi_logloss: 0.194717
[1200]	training's multi_logloss: 0.172303	valid_1's multi_logloss: 0.185202
[1300]	training's multi_logloss: 0.163

[I 2020-10-08 15:00:47,838] Trial 0 finished with value: 0.14405839759816663 and parameters: {'learning_rate': 0.0019104799438693207, 'num_iterations': 11552, 'n_estimators': 14216, 'num_leaves': 187, 'max_depth': 23, 'feature_fraction': 0.8083666515105918, 'bagging_fraction': 0.48700325072061146, 'bagging_freq': 4, 'min_child_samples': 5}. Best is trial 0 with value: 0.14405839759816663.


Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.358974	valid_1's multi_logloss: 0.371352
[200]	training's multi_logloss: 0.195842	valid_1's multi_logloss: 0.220445
[300]	training's multi_logloss: 0.135753	valid_1's multi_logloss: 0.17286
[400]	training's multi_logloss: 0.107355	valid_1's multi_logloss: 0.155702
[500]	training's multi_logloss: 0.0907272	valid_1's multi_logloss: 0.148274
[600]	training's multi_logloss: 0.0792036	valid_1's multi_logloss: 0.145372
[700]	training's multi_logloss: 0.0703014	valid_1's multi_logloss: 0.144149
[800]	training's multi_logloss: 0.0630271	valid_1's multi_logloss: 0.143433
Early stopping, best iteration is:
[798]	training's multi_logloss: 0.0631586	valid_1's multi_logloss: 0.143394
best_score defaultdict(<class 'collections.OrderedDict'>, {'training': OrderedDict([('multi_logloss', 0.06315861199997887)]), 'valid_1': OrderedDict([('multi_logloss', 0.14339449698927292)])})
multi logloss: 0.14339449698927

[I 2020-10-08 15:06:37,007] Trial 1 finished with value: 0.14339449698927292 and parameters: {'learning_rate': 0.00847889884258556, 'num_iterations': 14469, 'n_estimators': 13426, 'num_leaves': 485, 'max_depth': 28, 'feature_fraction': 0.9362914253071593, 'bagging_fraction': 0.5087573650975659, 'bagging_freq': 7, 'min_child_samples': 11}. Best is trial 1 with value: 0.14339449698927292.


Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.804665	valid_1's multi_logloss: 0.805159
[200]	training's multi_logloss: 0.671918	valid_1's multi_logloss: 0.673576
[300]	training's multi_logloss: 0.570582	valid_1's multi_logloss: 0.573372
[400]	training's multi_logloss: 0.491228	valid_1's multi_logloss: 0.495276
[500]	training's multi_logloss: 0.428035	valid_1's multi_logloss: 0.433293
[600]	training's multi_logloss: 0.377068	valid_1's multi_logloss: 0.383586
[700]	training's multi_logloss: 0.335536	valid_1's multi_logloss: 0.343231
[800]	training's multi_logloss: 0.301447	valid_1's multi_logloss: 0.310467
[900]	training's multi_logloss: 0.27324	valid_1's multi_logloss: 0.283587
[1000]	training's multi_logloss: 0.24974	valid_1's multi_logloss: 0.261419
[1100]	training's multi_logloss: 0.230049	valid_1's multi_logloss: 0.243068
[1200]	training's multi_logloss: 0.213423	valid_1's multi_logloss: 0.22782
[1300]	training's multi_logloss: 0.199

In [ ]:
print(study0.best_trial.params)